In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cftime
import warnings
xr.set_options(keep_attrs=True)
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../')
import my_utils as myf
import importlib
importlib.reload(myf)

<module 'my_utils' from '/gpfsm/dhome/laroach1/analysis/analysis_modelE-so-sst/published/processing/../my_utils.py'>

In [3]:
c6dir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/historical/'
sspdir = myf.cmip_dir+'ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp245/'

In [4]:
griddir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/piControl/r1i1p1f1/Ofx/'
garea = xr.open_dataset(griddir+'areacello/gn/v20180824/areacello_Ofx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc').areacello
ocnfr = xr.open_dataset(griddir+'sftof/gn/v20180824/sftof_Ofx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc').sftof
ocnfr = ocnfr/100.
garea = garea*ocnfr

In [5]:
def open_cmip(ens):
    ds = xr.open_mfdataset(c6dir+ens+'/Omon/mlotst/gn/*/mlotst_Omon_GISS-E2-1-G_historical_'+ens+'_*.nc')
    
    filestr = 'historical'
    if os.path.isdir(sspdir+ens):
        print('extend with SSP245')
        ds2 = xr.open_mfdataset(sspdir+ens+'/Omon/mlotst/gn/*/mlotst_Omon_GISS-E2-1-G_ssp245_'+ens+'_*.nc')
        ds = xr.concat([ds,ds2],dim='time').sel(time=slice('1979','2021'))
        filestr = 'hist+ssp245'
    ds['time'] = ds.indexes['time'].to_datetimeindex()
    ds['names'] = ens
    ds = ds.set_coords('names')
    ds = ds.sel(time=slice('1979','2021'))

    return ds, filestr

In [6]:
def get_mld_avg(ds, outfile):
          
    y1 = str(ds.time.values[0])[:4]
    y2 = str(ds.time.values[-1])[:4]

    ds1 = (ds*garea).sum(dim=('lat','lon'))/garea.sum(dim=('lat','lon'))
    ds1['region'] = 'Global'
    ds1 = ds1.set_coords('region') 

    mask = garea.copy()
    mask = mask.where(mask.lat<-45)
    ds2 = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds2['region'] = '45-90S'
    ds2 = ds2.set_coords('region')

    mask = garea.copy()
    mask = mask.where(mask.lat<-60)
    ds3 = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds3['region'] = '60-90S'
    ds3 = ds3.set_coords('region')

    mask = garea.copy()
    mask = mask.where(mask.lat<-50).where(mask.lat>-60)
    ds4 = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds4['region'] = '50-60S'
    ds4 = ds4.set_coords('region')
    
    mask = garea.copy()
    mask = mask.where(mask.lat<-45).where(mask.lat>-75)
    ds5 = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds5['region'] = '45-75S'
    ds5 = ds5.set_coords('region')
    
    mask = garea.copy()
    mask = mask.where(mask.lat<-50)
    ds6 = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds6['region'] = '50-90S'
    ds6 = ds6.set_coords('region')
    
    mask = garea.copy()
    mask = mask.where(mask.lat<-50).where(mask.lat>-65)
    ds7 = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds7['region'] = '50-65S'
    ds7 = ds7.set_coords('region')
    

    ds = xr.concat([ds1, ds2, ds3, ds4, ds5, ds6, ds7],dim='region')
    if 'type' in ds:
        ds = ds.drop('type')
    if 'time_bnds' in ds:
        ds = ds.drop('time_bnds')
    if 'lat_bnds' in ds:
        ds = ds.drop('lat_bnds')
    if 'lon_bnds' in ds:
        ds = ds.drop('lon_bnds')

    ds.to_netcdf(myf.processed_dir+'/timeseries/mlotst/'+outfile+str(y1)+'-'+str(y2)+'.nc')
    
    return ds

In [7]:
runmap = pd.read_csv('../e2.1runmap.csv',sep=',')
myens = runmap['names'].values

In [8]:
myens = [f for f in myens if  'f6' in f or 'f5' in f]

In [9]:
myens

['r1i1p1f5', 'r201i1p1f5', 'r202i1p1f5', 'r201i1p1f6', 'r202i1p1f6']

In [10]:
for ens in myens:
    ds, filestr = open_cmip(ens)
    ds = get_mld_avg(ds,'mlotst_avg_Omon_GISS-E2-1-G_'+filestr+'_'+ens+'_gn_')

extend with SSP245


In [11]:
argo_ml = xr.open_dataset(myf.obs_dir+'/ARGO/RG_argo_SO_ML_properties_v7.nc')
argo_ml = argo_ml.sel(thresh=0.03)
argo_ml = argo_ml.where(argo_ml.latitude<-50,drop=True)
# get area-weighted means
lat_r = [-65, -50]
lat_weights = np.cos(np.deg2rad(argo_ml.latitude))
lati = np.logical_and(argo_ml['latitude']>=lat_r[0], argo_ml['latitude']<=lat_r[-1])
argo_xymean = argo_ml.isel(latitude=lati).weighted(lat_weights).mean(dim=['latitude', 'longitude'])

ds = argo_xymean[['MLD']].rename({'MLD':'mlotst'})
ds.mlotst.attrs['units'] = 'm'
ds['region'] = '50-65S'
ds = ds.set_coords('region')
ds['names'] = 'OBS-ARGO'
ds = ds.set_coords('names')

y1 = str(ds.time.values[0])[:4]
y2 = str(ds.time.values[-1])[:4]
ds.to_netcdf(myf.processed_dir+'/timeseries/mlotst/mlotst_avg_Omon_OBS-ARGO_'+str(y1)+'-'+str(y2)+'.nc')